In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]


# Groups, Routes to Edges:

In [3]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [4]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])


In [5]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.539616936298,
  'Demand (eligible group, 1)': 47.77809012103125,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.564282762133406,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 115.15744798402402,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 134.75871572598555,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108034312373,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986336807,
  'Demand (eligible group, 1)': 7.64672374934271,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.411571393851563,
  'VoT (eligible g

In [6]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [7]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

# od_to_edges_array

In [8]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

# print(demand_array)
# VoT_array_base

In [9]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], 5))

T = 5
for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [10]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [11]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': 861.9885,
  'Latency (at bend)': 1.326448252,
  'Slope (after bend)': 0.000782666},
 'East Palo Alto': {'Flow (at bend)': 1001.517857,
  'Latency (at bend)': 2.213126553,
  'Slope (after bend)': 0.000584484},
 'Redwood City': {'Flow (at bend)': 881.1846667,
  'Latency (at bend)': 4.892192375,
  'Slope (after bend)': 0.001563724},
 'Belmont': {'Flow (at bend)': 1278.948125,
  'Latency (at bend)': 1.199911179,
  'Slope (after bend)': 0.001994138},
 'San Mateo': {'Flow (at bend)': 1034.092826,
  'Latency (at bend)': 5.541006284,
  'Slope (after bend)': 0.002147262},
 'Burlingame': {'Flow (at bend)': 845.15,
  'Latency (at bend)': 1.503111345,
  'Slope (after bend)': 0.000306601},
 'Millbrae': {'Flow (at bend)': 853.1818182,
  'Latency (at bend)': 2.384328452,
  'Slope (after bend)': 0.000321856}}

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [12]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [13]:
# np.array([1, 2, 3]) * np.array([6, 5, 3])

In [14]:
def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
                demand_array, VoT_array, num_el, od_to_edges_array, y, \
                coeff_input):

#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for e in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
            
    print()
    print("in_indices:", in_indices)
    print("el_indices:", el_indices)
    print()
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e] + coeff[1, e] * max(x[e, 0, t] - coeff[2, e], 0)
            ell[e, 1, t] = coeff[0, e] + coeff[1, e] * max(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0)

    obj_E = sum( y[(od, g, e, 0, t)] * VoT_array[od, g, t] * ell[e, 0, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 2, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                  for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in el_indices for t in range(T) )
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * ell[e, 1, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in in_indices for t in range(T) )
    obj_R = sum( y[(od, g, e, 0, t)] * tau[e, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) )

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I
    

# travel_times = {}

# for e in range(num_edges):
#     for t in range(T):
#         travel_times[(e, 0, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 0, t)] - coeff_input[2, e], 0.0)
#         travel_times[(e, 1, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)

#     obj_E[e] = sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
#                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
#                     + sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
#                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
#                     + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
#                           for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 

#     obj_I[e] = sum( argmin_y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
#                         for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
#                     + sum( argmin_y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
#                         for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#     obj_R[e] = sum( argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
#                     + sum( argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) )

#     obj[e] = lambda_E * obj_E[e] - lambda_R * obj_R[e] + lambda_I * obj_I[e]

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare, obj_E, obj_R, obj_I


# Latency, total, throughout the entire time horizon. 

def latency_total(T, num_edges, lambda_E, lambda_R, lambda_I, tau, \
                  demand_array, VoT_array, num_el, y, od_to_edges_array, \
                  coeff_input):
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges' second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for edge in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
    # In full:
    # y indices: (od, group, edge, "lane", time)

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            ell[e, 0, t] = coeff[0, e] + coeff[1, e] * max(x[e, 0, t] - coeff[2, e], 0)
            ell[e, 1, t] = coeff[0, e] + coeff[1, e] * max(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0)

    latency = sum( x[e, k, t] * ell[e, k, t] \
                for e in range(num_edges) for k in range(2) for t in range(T) )

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return latency

In [15]:
# arr = np.arange(5)
# arr.reshape((5, 1))

In [16]:
def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0, B_min = 0.1):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
    assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
#                     for od in range(len(od_to_edges_list_full))]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

#     print()
#     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
#     print()

    B_max_list = [sum(tau_feas.value[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                            for od in range(len(od_to_edges_list_full))]
#     B_max_wrt_tau_feas = min([B_max for B_max in B_max_list if B_max >= B_min])
    B_max_wrt_tau_feas = max(B_max_list)
    B_feas = min(B, B_max_wrt_tau_feas)
    
    print()
    print("B:", B)
    print("B_max_wrt_tau_feas:", B_max_wrt_tau_feas)
    print("B_feas:", B_feas)
    print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas, 0.0), decimals=4)
    
#     return tau_feas.value, B_feas



In [17]:
# # OLD def proj_tau_B:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0):

# #     print()
# #     print("tau.shape[0]:", tau.shape[0])
# #     print("num_edges:", num_edges)
# #     print()

#     assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
#     assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
#     tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
#     func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

#     objective = cp.Minimize(func)

#     constraints = []
#     constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
#     constraints += [tau_feas <= tau_max.reshape((num_edges, 1)) * np.ones((1, T))]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od]) \
#                     for od in range(len(od_to_edges_list_full)) for t in range(T)]
    
#     prob = cp.Problem(objective, constraints)
#     result = prob.solve()

# #     print()
# #     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
# #     print()

#     return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
#             np.round(np.maximum(B_feas.value, 0.0), decimals=4)

# Chinmay's Algorithm:

## Convex Program for CBCP and DBCP Equilibria:

In [18]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [19]:
# num_edges = 8
# od_to_edges_array = np.array([[0, 0], [0, 3], [0, 7], [1, 5], [1, 7], [3, 3], [3, 6]])

# od_to_edges_list_full = [list(range(od_to_edges_array[od, 0], od_to_edges_array[od, 1] + 1)) \
#                       for od in range(od_to_edges_array.shape[0])]

# edge_to_od = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
#             for e in range(num_edges)]

# print(od_to_edges_list_full)
# print()
# print(edge_to_od)

In [20]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
                      demand_array, VoT_array, num_el, coeff_input):
    
#     print("tau (in solve_CBCP_direct):", tau)
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T."
    assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have identical shape."
    assert np.all(demand_array > 0.0), "Each entry of demand_array must be strictly positive."
    assert np.all(tau >= 0.0), "Each entry of tau must be non-negative."
    assert num_el <= demand_array.shape[1], "num_el, the number of eligible income groups, should not exceed \
                                            demand_array.shape[1], which should equal the number of income groups."
    
    num_groups = demand_array.shape[1]
    
    ## Variable indices:
    # y indices: (od, income group, edge, "lane", time)
    # y indices: (od, income group, edge, "lane", time)

    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    group_indices = list(range(num_groups))
    
    od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                          for od in range(od_to_edges_array.shape[0])]
    
    edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
                   for e in range(num_edges)]
    
    num_od = len(od_to_edges_list_full)
    
    # Variables:
    y = {}
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
                for g in in_indices:
                    for k in [0, 1]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
    
    x = {}
    for e in range(num_edges):
        for k in [0, 1]:
            for t in range(T):
                x[(e, k, t)] = cp.Variable(1)

    # Objective:
    func = 0.0
    for e in range(num_edges):
        for t in range(T):
            func += coeff[0, e] * x[e, 0, t] \
                    + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[e, 0, t] - coeff[2, e], 0))
            func += coeff[0, e] * x[e, 1, t] \
                    + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[e, 1, t]/num_gp_lanes - coeff[2, e], 0))
    
#     print("VoT_array.shape:", VoT_array.shape)
#     print("el_indices:", el_indices)
#     print("in_indices:", in_indices)
    
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
                for g in in_indices:
                    func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in el_indices  \
                    for k in [0, 1, 2] for t in range(T)]
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in in_indices  \
                    for k in [0, 1] for t in range(T)]
    constraints += [x[(e, k, t)] >= 0.0 for e in range(num_edges) \
                    for k in [0, 1] for t in range(T)]
    
#     print()
#     print("demand_array:", demand_array)
#     print()
#     print("B:", B)
#     print()
#     print("el_indices:", el_indices)
#     print()
#     print("in_indices:", in_indices)
#     print()
#     print("edge_to_ods:", edge_to_ods)
#     print()
#     print("od_to_edges_list_full:", od_to_edges_list_full)
#     print()
    
    od_g_e_t_list = []

    for e in range(num_edges):
        for t in range(T):
            
            ## Edge contraints:
            constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 0, t)] ]
            constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 1, t)] ]

            ## Group flow constraints:
            for od in edge_to_ods[e]:
                for g in el_indices:
#                     print("(od, g, e, t):", od, g, e, t)
                    assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
                    od_g_e_t_list += [(od, g, e, t)]
        
#                     if (od, g, e, t) == (0, 0, 0, 0):
#                     if (e, t) == (2, 0) or (2, 1) or (2, 3):
#                         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g]]
                    
#                 for g in in_indices:
# #                     print("(od, g, e, t):", od, g, e, t)
#                     assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
#                     od_g_e_t_list += [(od, g, e, t)]
                    
#                     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g]]

            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in el_indices]
            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in in_indices]
    
    constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                    <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
#     result = prob.solve(solver = cp.MOSEK, verbose = True)
    result = prob.solve(solver = cp.MOSEK)
    
#     for variable in prob.variables():
#         print("Variable %s" % variable.name())
    
    assert prob.status != "infeasible", "problem.status should not be infeasible."
    assert prob.status != "unbounded", "problem.status should not be unbounded."
    print()
    print("prob.status:", prob.status)

    # Extract Values:
    y_values = {}
    for e in range(num_edges):
        for od in edge_to_ods[e]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
                for g in in_indices:
                    for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)

    return y_values


## <font color='blue'>Implement Zeroth-order Gradient Descent:</font> 

In [21]:
## Functions defined above:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max = 1.0, B_max = 1.0):

# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):

#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):

In [22]:
# coeff_input
# demand_array.shape

In [23]:

T = 5

# num_iters = 1000
# num_iters = 100
num_iters = 200
# num_iters = 3

num_el = 2
num_groups = demand_array.shape[1]

assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T"
assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have the same shape."

num_ods = demand_array.shape[0]
group_indices = list(range(demand_array.shape[1]))
# num_edges = 6
num_edges = 7
num_gp_lanes = 3

# tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
# assert tau.shape[0] == num_edges, "tau must have a num_edges number of entries."

# B = 0.8
a = np.array([1E-3, 1E-3, 0.0, 0.0, 0.0])

error_bound = 1E-2
diffs_num_cols = 5

demand_edges_array = np.zeros(num_edges)

# demand_array_temp = np.ones(demand_array.shape)
# demand_array_temp = np.random.uniform(low=0.05, high=0.5, size=demand_array.shape)
demand_array_temp = demand_array


# TODO: Modify coeff_input:

## coeff_input: const, slope, x-coordinate of transition point

# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))


coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]


# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):
# Already defined: T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array

    
# print("od_to_edges_array[1, 0]:", int(od_to_edges_array[1, 0]))
# print("range(od_to_edges_array[1, 0], od_to_edges_array[1, 1] + 1):",\
#      range(int(od_to_edges_array[1, 0]), int(od_to_edges_array[1, 1]) + 1))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                         for od in range(num_ods)]
edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
               for e in range(num_edges)]
for e in range(num_edges):
    demand_edges_array[e] = sum([np.sum(demand_array_temp[od, :]) for od in range(num_ods) \
                                 if od in edge_to_ods[e]])

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

tau_max_from_latency = np.zeros((num_edges, T))
VoT_max_el = np.zeros((num_edges, T))
for e in range(num_edges):
    for t in range(T):
        VoT_max_el[e, t] = max([VoT_array[od_index, group_index, t] for od_index in edge_to_ods[e] \
                                for group_index in el_indices])
        tau_max_from_latency[e, t] = VoT_max_el[e, t] * (latency_max(demand_edges_array[e], coeff_input[:, e]) - coeff_input[0, e]) \

tau_max_monetary_value = 15.0
# fraction_tau_max = tau_max_monetary_value / np.max(tau_max_from_latency)

tau_upper_limit = np.minimum(tau_max_from_latency, tau_max_monetary_value)
B_upper_limit = max([sum(tau_upper_limit[e, t] for e in od_to_edges_list_full[od] for t in range(T) ) \
                     for od in range(len(od_to_edges_list_full))])

print("tau_upper_limit:", tau_upper_limit)
print()
print("B_upper_limit:", B_upper_limit)

d = num_edges * T + 1


tau = np.zeros((num_edges, T, num_iters))
tau_perturbed = np.zeros((num_edges, T, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)

fraction_tau_max = 0.5
fraction_B_max_1 = 0.25
fraction_B_max_2 = 0.5
assert fraction_B_max_1 < fraction_B_max_2, "We must have fraction_B_max_1 < fraction_B_max_2."

# tau[:, :, 0] = fraction_tau_max * tau_upper_limit.reshape((num_edges, 1)) @ np.ones((1, T))
tau[:, :, 0] = np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit * fraction_tau_max, tau_upper_limit)

# B[0] = fraction_B_max * B_upper_limit
# B_upper_limit_init = max([sum(tau[e, t, 0] for e in od_to_edges_list_full[od] for t in range(T) ) \
#                      for od in range(len(od_to_edges_list_full))])
B_upper_limit_init = sum(tau[e, t, 0] for e in range(num_edges) for t in range(T) )

B[0] = np.random.triangular(0.0, fraction_B_max_1 * B_upper_limit_init, fraction_B_max_2 * B_upper_limit_init)

print()
print("tau[:, :, 0]:", tau[:, :, 0])
print()
print("np.sum(tau[:, :, 0]):", np.sum(tau[:, :, 0]))
print()
print("B_upper_limit_init:", B_upper_limit_init)
print()
print("B[0]:", B[0])

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1 )) \
                         for od in range(od_to_edges_array.shape[0])]


# # To disable when restarting from scratch
# tau[:, :, 0] = tau_next_init
# B[0] = B_next_init
# eta[0] = eta_bar * (index_next_init+1)**(-1/2) * d**(-1)
# delta[0] = delta_bar * (index_next_init+1)**(-1/4) * d**(-1/2)

tau_upper_limit: [[0.31942364 0.31942364 0.31942364 0.31942364 0.31942364]
 [0.24983105 0.24983105 0.24983105 0.24983105 0.24983105]
 [0.99953381 0.99953381 0.99953381 0.99953381 0.99953381]
 [1.02805916 1.02805916 1.02805916 1.02805916 1.02805916]
 [1.6042613  1.6042613  1.6042613  1.6042613  1.6042613 ]
 [0.19224693 0.19224693 0.19224693 0.19224693 0.19224693]
 [0.21782897 0.21782897 0.21782897 0.21782897 0.21782897]]

B_upper_limit: 23.055924301525522

tau[:, :, 0]: [[0.15857265 0.21745607 0.18110781 0.05420639 0.12326287]
 [0.08330864 0.16125029 0.10843942 0.11692869 0.15816359]
 [0.49155991 0.45658858 0.81556112 0.56199594 0.17310703]
 [0.47933574 0.41249396 0.53453631 0.62914512 0.51000407]
 [1.37659514 0.51490456 0.86467184 0.24133247 0.79522482]
 [0.1116855  0.10802463 0.06768341 0.10197967 0.09781088]
 [0.0830568  0.12265723 0.09556692 0.12075116 0.14251909]]

np.sum(tau[:, :, 0]): 11.271488324617103

B_upper_limit_init: 11.271488324617101

B[0]: 4.36625577848879


In [32]:
time_1 = time.time()

welfare_list = []

## Set lambda:

lambda_E, lambda_R, lambda_I = 2.0, 2.0, 5.0

delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 1E-5
delta_bar = 0.01

for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(d)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
#     print("w_i:", w_i)
    tau_perturbed[:, :, i] = tau[:, :, i] + delta[i] * w_i[:-1].reshape((num_edges, T))
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, :, i] < B_perturbed[i] or tau_perturbed[:, :, i] < 0 or B_perturbed[i] < 0:

    print("i:\n", i)
    print("tau[:, :, i], before projection:\n", tau[:, :, i])
    print("B[i], before projection:\n", B[i])
    print("tau_perturbed[:, :, i], before projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], before projection:\n", B_perturbed[i])

    tau_perturbed[:, :, i], B_perturbed[i] = proj_tau_B(T, num_edges, tau_perturbed[:, :, i], B_perturbed[i], \
                                                        od_to_edges_list_full, \
                                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i], after projection:\n", tau[:, :, i])
    print("B[i], after projection:\n", B[i])
    print("tau_perturbed[:, :, i], after projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], after projection:\n", B_perturbed[i])

    # TODO: Remove "network":

    y_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                 tau[:, :, i], B[i], od_to_edges_array, \
                                 demand_array_temp, VoT_array, num_el, coeff_input)

    y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                           tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
                                           demand_array_temp, VoT_array, num_el, coeff_input)
    
#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input):

    welfare, obj_E, obj_R, obj_I = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                                               tau[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                                               y = y_values, coeff_input = coeff_input)
    
    welfare_perturbed, obj_E_perturbed, obj_R_perturbed, obj_I_perturbed \
        = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                      tau_perturbed[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                      y = y_perturbed_values, coeff_input = coeff_input)
    
    print("welfare:", welfare)
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print()
    print("welfare_perturbed:", welfare_perturbed)
    print("obj_E_perturbed:", obj_E_perturbed)
    print("obj_R_perturbed:", obj_R_perturbed)
    print("obj_I_perturbed:", obj_I_perturbed)
    print()
    
    welfare_list.append(welfare)
    
    tau[:, :, i+1] = tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((num_edges, T)) \
                        * (welfare_perturbed - welfare)
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    
    print("tau[:, :, i+1], before projection:\n", tau[:, :, i+1])
    print("B[i+1], before projection:\n", B[i+1])
    
    tau[:, :, i+1], B[i+1] = proj_tau_B(T, num_edges, tau[:, :, i+1], B[i+1], od_to_edges_list_full, \
                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i+1], after projection:\n", tau[:, :, i+1])
    print("B[i+1], after projection:\n", B[i+1])
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.max(np.absolute(tau[:, :, i-diffs_num_cols : i-1] - tau[:, :, i-diffs_num_cols+1 : i]), axis = 2)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:\n", tau_diffs)
        print("B_diffs:\n", B_diffs)
        print()
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            print("Within error bound.")
            break

    time_2 = time.time()

    print()
    print("Iteration count:", i)
    print("Time:", time_2 - time_1)
    print()




Iter: 0
i:
 0
tau[:, :, i], before projection:
 [[0.15857265 0.21745607 0.18110781 0.05420639 0.12326287]
 [0.08330864 0.16125029 0.10843942 0.11692869 0.15816359]
 [0.49155991 0.45658858 0.81556112 0.56199594 0.17310703]
 [0.47933574 0.41249396 0.53453631 0.62914512 0.51000407]
 [1.37659514 0.51490456 0.86467184 0.24133247 0.79522482]
 [0.1116855  0.10802463 0.06768341 0.10197967 0.09781088]
 [0.0830568  0.12265723 0.09556692 0.12075116 0.14251909]]
B[i], before projection:
 4.36625577848879
tau_perturbed[:, :, i], before projection:
 [[0.15845704 0.21755335 0.18110079 0.05421957 0.12324554]
 [0.08337217 0.16139799 0.10838812 0.11684784 0.15781698]
 [0.49178781 0.45658594 0.81558153 0.56210046 0.17306416]
 [0.47934901 0.41223328 0.53393456 0.62892152 0.51017581]
 [1.37707813 0.51459481 0.86478148 0.24143777 0.79454361]
 [0.1114753  0.10836897 0.06796224 0.10173049 0.09696094]
 [0.08274063 0.12277924 0.0953315  0.1207217  0.1424738 ]]
B_perturbed[i], before projection:
 4.366156629452


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 4010896.9870696487
obj_E: 5844.054697529218
obj_R: 6934.653517540934
obj_I: 802615.6369419345

welfare_perturbed: 4010848.894487799
obj_E_perturbed: 5844.122886950301
obj_R_perturbed: 6936.109711952742
obj_I_perturbed: 802606.5736275607

tau[:, :, i+1], before projection:
 [[0.18928722 0.15240957 0.1963107  0.06760988 0.05530116]
 [0.10077444 0.17361487 0.08955575 0.13620051 0.12762929]
 [0.4002622  0.477882   0.76684069 0.57010834 0.07516163]
 [0.48655932 0.44877133 0.51089651 0.656092   0.56079479]
 [1.34592951 0.6008431  0.91140154 0.25323928 0.85485168]
 [0.12966233 0.08486746 0.09315403 0.2364091  0.18722409]
 [0.1151538  0.13327178 0.19756971 0.17893791 0.18395585]]
B[i+1], before projection:
 4.416622220361376

B: 4.416622220361376
B_max_wrt_tau_feas: 11.704381398346905
B_feas: 4.416622220361376

tau[:, :, i+1], after projection:
 [[0.1893 0.


prob.status: optimal

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 3981248.217239634
obj_E: 5705.833687051221
obj_R: 748.3485470261255
obj_I: 794266.6493919168

welfare_perturbed: 4120029.742429777
obj_E_perturbed: 5562.957110019576
obj_R_perturbed: 748.7942896216997
obj_I_perturbed: 822080.2833577963

tau[:, :, i+1], before projection:
 [[  71.89797482    3.01710766  141.31107272  -16.85275228   73.97781429]
 [  43.81052853  165.5272181   -24.38600051  144.42356312  -64.08204688]
 [ -79.03146695   81.45954975   21.10340832    4.90736792  -76.88534099]
 [ 244.87406628   37.13844888  -65.81297793   11.19793846   49.26376637]
 [  80.01330267  -71.33800314  -46.3410743   -68.71217718  -72.23886957]
 [  43.21214434   45.16050921  -51.81285644 -127.89276287  132.80705007]
 [-104.30732455   27.47230481   -3.65133882    2.82053036 -109.12834267]]
B[i+1], before projection:
 -21.26045569970573

B: -21.26045569970573
B_max_

KeyboardInterrupt: 

In [25]:
# dict_latency_params["Belmont"]

In [33]:
min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, :, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

# print("argmin_B:", argmin_B)
# print("B[argmin_welfare_list]:", B[argmin_welfare_list])
# print("min_welfare:", min_welfare)
# print()
print(welfare_list)

# B
print()
print(min_welfare)
print()
print(argmin_welfare_list)
print()
print("len(welfare_list)", len(welfare_list))
print()
print("argmin_B:", argmin_B)
print()
print("argmin_tau:", argmin_tau)
print()

# welfare_list[argmin_welfare_list]
# B[argmin_welfare_list]

[4013414.620835087, 4012138.6925119585, 4011135.4434486874, 4010896.9870696487, 4006512.782672831, 4018246.854865534, 3981248.217239634]

3981248.217239634

6

len(welfare_list) 7

argmin_B: 7.4821

argmin_tau: [[0.     0.     0.     0.     0.3194]
 [0.     0.     0.2498 0.     0.2498]
 [0.     0.0943 0.     0.849  0.6524]
 [0.     1.0281 0.     1.0281 0.    ]
 [0.     0.6123 1.6043 0.     0.    ]
 [0.1922 0.     0.     0.1922 0.1922]
 [0.2178 0.     0.     0.     0.    ]]



In [34]:
# index_next_init = argmin_welfare_list
# B_next_init = B[index_next_init]
# tau_next_init = tau[:, :, index_next_init]

# print("index_next_init:", index_next_init)
# print("B_next_init:", B_next_init)
# print("tau_next_init:", tau_next_init)


In [184]:
## Initialize lambda:

lambda_E, lambda_R, lambda_I = 2.0, 5.0, 5.0

## Initialize tau, alpha values:

filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

directory_inits = '../data/opt_tolls_subsidies_metrics/'
df_inits = pd.read_csv(directory_inits + 'opt_CBCP___' + filename_segment + '.csv')

print("filename_segment:", filename_segment)
print()

inits_tau_arr_as_object = df_inits.to_numpy()[:, 1:6]
inits_B_arr_as_object = df_inits.to_numpy()[0, 7]

argmin_tau = np.zeros((num_edges, T))
argmin_B = 0

for e in range(num_edges):
    for t in range(T):
        argmin_tau[e, t] = inits_tau_arr_as_object[e, t]
        argmin_B = inits_B_arr_as_object

print(argmin_tau)
print()
print(argmin_B)


filename_segment: 2_5_5

[[0.23 0.32 0.27 0.   0.32]
 [0.   0.25 0.25 0.09 0.11]
 [1.   0.   0.84 0.82 0.35]
 [0.09 0.   0.   0.05 0.3 ]
 [0.05 0.16 0.04 1.6  1.24]
 [0.19 0.13 0.19 0.19 0.  ]
 [0.   0.08 0.   0.09 0.  ]]

0.0


In [185]:

# argmin_tau = np.array([[0.0, 0.32, 0.32, 0.0, 0.04], \
#                        [0.04, 0.07, 0.21, 0.06, 0.15], \
#                        [0.79, 0.66, 0.98, 0.07, 0.79], \
#                        [0.54, 0.8, 0.43, 0.24, 0.3], \
#                        [0.74, 0.46, 1.2, 1.0, 0.39], \
#                        [0.04, 0.13, 0.1, 0.07, 0.08], \
#                        [0.15, 0.14, 0.11, 0.1, 0.14]])

# argmin_B = 3.65

# VoT_array[:, :, 0]

In [186]:
argmin_y = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                    argmin_tau, argmin_B, od_to_edges_array, \
                                    demand_array_temp, VoT_array, num_el, coeff_input)

print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)
print("min_welfare:", min_welfare)
print()
print("argmin_y:\n")

argmin_y



prob.status: optimal
argmin_tau: [[0.23 0.32 0.27 0.   0.32]
 [0.   0.25 0.25 0.09 0.11]
 [1.   0.   0.84 0.82 0.35]
 [0.09 0.   0.   0.05 0.3 ]
 [0.05 0.16 0.04 1.6  1.24]
 [0.19 0.13 0.19 0.19 0.  ]
 [0.   0.08 0.   0.09 0.  ]]
argmin_B: 0.0
min_welfare: 3981248.217239634

argmin_y:



{(0, 0, 0, 0, 0): -0.0,
 (0, 0, 0, 1, 0): -0.0,
 (0, 0, 0, 2, 0): 47.77809012103125,
 (0, 1, 0, 0, 0): -0.0,
 (0, 1, 0, 1, 0): 3.816936697283444e-05,
 (0, 1, 0, 2, 0): 27.564244592766432,
 (0, 2, 0, 0, 0): 9.496428455423682e-05,
 (0, 2, 0, 1, 0): 115.15735301973946,
 (0, 3, 0, 0, 0): 0.0002863065410617807,
 (0, 3, 0, 1, 0): 134.7584294194445,
 (0, 4, 0, 0, 0): 0.003637300495045737,
 (0, 4, 0, 1, 0): 287.2774430426287,
 (0, 0, 0, 0, 1): -0.0,
 (0, 0, 0, 1, 1): -0.0,
 (0, 0, 0, 2, 1): 47.77809012103125,
 (0, 1, 0, 0, 1): -0.0,
 (0, 1, 0, 1, 1): 2.7312181481068514e-05,
 (0, 1, 0, 2, 1): 27.564255449951926,
 (0, 2, 0, 0, 1): 9.703331910854779e-05,
 (0, 2, 0, 1, 1): 115.15735095070491,
 (0, 3, 0, 0, 1): 0.00017548058307376963,
 (0, 3, 0, 1, 1): 134.7585402454025,
 (0, 4, 0, 0, 1): 0.0021230446626202266,
 (0, 4, 0, 1, 1): 287.27895729846114,
 (0, 0, 0, 0, 2): -0.0,
 (0, 0, 0, 1, 2): -0.0,
 (0, 0, 0, 2, 2): 47.77809012103125,
 (0, 1, 0, 0, 2): -0.0,
 (0, 1, 0, 1, 2): 3.2538396612477404e-05,
 

In [187]:
# demand_array_temp

# tau_upper_limit

# B_upper_limit

# y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
#                                            tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
#                                            demand_array_temp, VoT_array, num_el, coeff_input)

# T
# num_edges
# num_gp_lanes
# B_perturbed[i]
# coeff_input
# od_to_edges_array
# demand_array_temp
# VoT_array
# num_el
# coeff_input

# np.minimum(np.array([1, 2]), 0.5)

# tau_upper_limit
# argmin_welfare_list

# Compute Cost Metrics

In [188]:
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

edge_to_od_dict = {}
for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

argmin_x = {}
for e in range(num_edges):
    for t in range(T):
        argmin_x[(e, 0, t)] = 0
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_x[(e, 1, t)] = 0
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 2, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)

argmin_y_in_el_total = {}      
for e in range(num_edges):
    for t in range(T):
        argmin_y_in_el_total[(e, 0, t, 'el')] = sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'el')] = sum(argmin_y[(od, g, e, 2, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 0, t, 'in')] = sum(argmin_y[(od, g, e, 0, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'in')] = sum(argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)


# y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] for g in el_indice

# argmin_x

argmin_y_in_el_total

{(0, 0, 0, 'el'): 4.9705197929315095e-05,
 (0, 1, 0, 'el'): 596.3166970912687,
 (0, 0, 0, 'in'): 872.5209597887523,
 (0, 1, 0, 'in'): 3379.2659259063785,
 (0, 0, 1, 'el'): 3.563192767234778e-05,
 (0, 1, 1, 'el'): 596.3167111645389,
 (0, 0, 1, 'in'): 525.3929137217343,
 (0, 1, 1, 'in'): 3726.393971973397,
 (0, 0, 2, 'el'): 4.23871422638701e-05,
 (0, 1, 2, 'el'): 596.3167044093243,
 (0, 0, 2, 'in'): 693.4374287662333,
 (0, 1, 2, 'in'): 3558.349456928898,
 (0, 0, 3, 'el'): 0.0,
 (0, 1, 3, 'el'): 596.3167467964665,
 (0, 0, 3, 'in'): 1002.0020930978651,
 (0, 1, 3, 'in'): 3249.7847925972656,
 (0, 0, 4, 'el'): 3.5660696603874235e-05,
 (0, 1, 4, 'el'): 596.31671113577,
 (0, 0, 4, 'in'): 474.9394462638244,
 (0, 1, 4, 'in'): 3776.8474394313066,
 (1, 0, 0, 'el'): 0.0,
 (1, 1, 0, 'el'): 658.9583981680674,
 (1, 0, 0, 'in'): 1118.3317709401786,
 (1, 1, 0, 'in'): 3396.940678112125,
 (1, 0, 1, 'el'): 9.12292511921705e-05,
 (1, 1, 1, 'el'): 658.9583069388162,
 (1, 0, 1, 'in'): 352.9109127345051,
 (1, 1

In [189]:
# coeff_input

print("argmin_tau[5, 3]:", argmin_tau[5, 3])
print("argmin_x[(5, 0, 3)]:", argmin_x[(5, 0, 3)])
print("argmin_x[(5, 1, 3)]:", argmin_x[(5, 1, 3)])


argmin_tau[5, 3]: 0.19
argmin_x[(5, 0, 3)]: 876.1453441266844
argmin_x[(5, 1, 3)]: 5768.9253791290075


In [190]:
## coeff_input: const, slope, x-coordinate of transition point
# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))

travel_times = {}

for e in range(num_edges):
    for t in range(T):
        travel_times[(e, 0, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 0, t)] - coeff_input[2, e], 0.0)
        travel_times[(e, 1, t)] = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)

edge_demand = {}
avg_travel_time = {}
percent_on_express = {}
obj_E = np.zeros(num_edges)
obj_I = np.zeros(num_edges)
obj_R = np.zeros(num_edges)
obj = np.zeros(num_edges)

for e in range(num_edges):
    avg_travel_time[e, 'el'] = 0.0
    avg_travel_time[e, 'in'] = 0.0
    avg_travel_time[e, 'ex'] = 0.0
    avg_travel_time[e, 'gp'] = 0.0
    
    percent_on_express[e, 'el'] = 0.0
    percent_on_express[e, 'in'] = 0.0
    percent_on_express[e, 'all'] = 0.0
    
    obj_E[e] = 0.0
    obj_I[e] = 0.0
    obj_R[e] = 0.0
    obj[e] = 0.0
    
    for t in range(T):
        edge_demand[e, t, 'el'] = sum(argmin_y_in_el_total[(e, k, t, 'el')] for k in range(2))
        edge_demand[e, t, 'in'] = sum(argmin_y_in_el_total[(e, k, t, 'in')] for k in range(2))
        
    percent_on_express[e, 'el'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))
    percent_on_express[e, 'in'] += sum(argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))
    percent_on_express[e, 'all'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] + edge_demand[e, t, 'in'] for t in range(T))
    
    avg_travel_time[e, 'el'] += sum(argmin_y_in_el_total[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))    
    avg_travel_time[e, 'in'] += sum(argmin_y_in_el_total[(e, k, t, 'in')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))    
#     avg_travel_time[e, 'ex'] += sum( (argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T))
#     avg_travel_time[e, 'gp'] += sum( (argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')] for t in range(T))
    avg_travel_time[e, 'ex'] += sum(travel_times[(e, 0, t)] for t in range(T)) / T 
    avg_travel_time[e, 'gp'] += sum(travel_times[(e, 1, t)] for t in range(T)) / T 
    
    obj_E[e] = sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                          for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 

    obj_I[e] = sum( argmin_y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

    obj_R[e] = sum( argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) )

    obj[e] = lambda_E * obj_E[e] - lambda_R * obj_R[e] + lambda_I * obj_I[e]


# welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, argmin_tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input)

# avg_travel_time
# percent_on_express
# obj_R



In [191]:
# e = 0
# od = 0
# g = 0
# t = 0

# # sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times[e, 0, t] \
# #                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times[e, 0, t] + tau[e, t]) \
#     for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# #                     + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g, t] * travel_times[e, 1, t] \
# #                           for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) ) 

# # argmin_y[(od, g, e, 1, t)]



# # sum(edge_demand[e, t, 'el'] for t in range(T) )

In [192]:
# travel_times
argmin_x

{(0, 0, 0): 872.5210094939503,
 (0, 1, 0): 3975.5826229976474,
 (0, 0, 1): 525.392949353662,
 (0, 1, 1): 4322.710683137936,
 (0, 0, 2): 693.4374711533756,
 (0, 1, 2): 4154.666161338222,
 (0, 0, 3): 1002.0020930978651,
 (0, 1, 3): 3846.1015393937323,
 (0, 0, 4): 474.939481924521,
 (0, 1, 4): 4373.164150567077,
 (1, 0, 0): 1118.3317709401786,
 (1, 1, 0): 4055.8990762801927,
 (1, 0, 1): 352.9110039637563,
 (1, 1, 1): 4821.319843256615,
 (1, 0, 2): 323.3396969008672,
 (1, 1, 2): 4850.891150319506,
 (1, 0, 3): 1043.4621270547589,
 (1, 1, 3): 4130.768720165613,
 (1, 0, 4): 1033.054972490728,
 (1, 1, 4): 4141.175874729643,
 (2, 0, 0): 907.4679198011111,
 (2, 1, 0): 5886.250010383692,
 (2, 0, 1): 1208.0878332261354,
 (2, 1, 1): 5585.630096958665,
 (2, 0, 2): 948.2667181745497,
 (2, 1, 2): 5845.451212010253,
 (2, 0, 3): 953.0276856881419,
 (2, 1, 3): 5840.69024449666,
 (2, 0, 4): 1106.6037225574453,
 (2, 1, 4): 5687.114207627356,
 (3, 0, 0): 1350.5284793603516,
 (3, 1, 0): 4697.109819740149,
 (

In [193]:
# print("argmin_x[(3, 0, 0)]:", argmin_x[(3, 0, 0)])
# print("argmin_x[(3, 1, 0)]:", argmin_x[(3, 1, 0)])

In [194]:
opt_data_array = np.zeros((num_edges, 16))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau

# argmin_tau_avg
opt_data_array[:, 5] = np.mean(argmin_tau, axis=1)

# B (as array)
opt_data_array[:, 6] = argmin_B * np.eye(1, num_edges, 0)

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 7] = np.array([percent_on_express[e, 'all'] for e in range(num_edges)]) * 100
opt_data_array[:, 8] = np.array([percent_on_express[e, 'el'] for e in range(num_edges)]) * 100
opt_data_array[:, 9] = np.array([percent_on_express[e, 'in'] for e in range(num_edges)]) * 100

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 10] = np.array([avg_travel_time[e, 'ex'] for e in range(num_edges)])
opt_data_array[:, 11] = np.array([avg_travel_time[e, 'gp'] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 12] = np.array([obj_E[e] for e in range(num_edges)]) 
opt_data_array[:, 13] = np.array([obj_I[e] for e in range(num_edges)]) 
opt_data_array[:, 14] = np.array([obj_R[e] for e in range(num_edges)]) 
opt_data_array[:, 15] = np.array([obj[e] for e in range(num_edges)]) 


opt_data_array = np.round(opt_data_array, decimals=2)

In [195]:
# 

In [196]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)", \
                 "B", \
                 "% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

df_opt_save



,tau (t=1),tau (t=2),tau (t=3),tau (t=4),tau (t=5),tau (time-averaged),B,% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
e=1,0.23,0.32,0.27,0.00,0.32,0.23,0.0,14.72,0.0,16.78,1.35,1.73,297.88,42056.18,708.01,207336.59
e=2,0.00,0.25,0.25,0.09,0.11,0.14,0.0,14.96,0.0,17.15,2.24,2.49,500.70,60592.85,376.61,302082.62
e=3,1.00,0.00,0.84,0.82,0.35,0.60,0.0,15.08,0.0,17.22,5.12,6.52,1757.97,201343.84,2872.81,995871.12
e=4,0.09,0.00,0.00,0.05,0.30,0.09,0.0,22.34,0.0,25.47,1.34,1.77,427.38,46430.81,579.98,230108.88
e=5,0.05,0.16,0.04,1.60,1.24,0.62,0.0,16.07,0.0,18.25,6.06,8.09,2534.58,277184.16,3449.03,1373744.79
e=6,0.19,0.13,0.19,0.19,0.00,0.14,0.0,14.28,0.0,16.14,1.53,1.83,459.61,55019.54,620.95,272912.17
e=7,0.00,0.08,0.00,0.09,0.00,0.03,0.0,16.44,0.0,18.57,2.49,2.75,747.60,89096.52,185.10,446052.34


In [197]:
# print(min_welfare)
# print(np.sum(obj))

In [198]:
# # e = 0
# # t = 4

# e = 4
# t = 3

# print("travel_times[(e, 0, t)]:\n", travel_times[(e, 0, t)])
# print()
# print("travel_times[(e, 0, t)] + argmin_tau[(e, t)] / np.max(VoT_array[:, in_indices, t]):\n", \
#       travel_times[(e, 0, t)] + argmin_tau[(e, t)] / np.max(VoT_array[:, in_indices, t]) )
# print()
# print("travel_times[(e, 1, t)]:\n", travel_times[(e, 1, t)])
# print()
# print("Total ineligible user express lane flow at (e, t):\n", argmin_y_in_el_total[(e, 0, t, "in")])

# VoT_array[:, in_indices, t]
# # demand_array[:, in_indices]

# # print("")

# obj_R

# for e in range(num_edges):
#     for t in range(T):
#         print(sum(argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in in_indices) \
#                     + sum(argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in el_indices) )

# VoT_array[:, :, 0]

In [199]:
random_string = ""
for idx_rand_str in range(10):
    random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_tolls_subsidies_metrics/"
random_filename = "opt_CBCP___" + random_string + '.csv'

df_opt_save.to_csv(directory_to_save + random_filename)

# Compute good initializations for alpha:

In [ ]:
# # el_indices
# # in_indices

# e = 0
# # VoT_array[edge_to_od_dict[e], in_indices, :]

# VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :]
# VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices, :]

# # print("VoT_array[:, in_indices, :]:\n", VoT_array[:, in_indices, :])
# # print()
# # print("VoT_array[edge_to_od_dict[e], :, :]:\n", VoT_array[edge_to_od_dict[e], :, :])

# # VoT_array_trunc_in

# demand_array.shape

In [ ]:
# arr_temp = np.stack([np.array([1, 2, 3, 4, 5]), np.array([3, 1, 4, 5, 2]), np.array([2, 5, 1, 4, 3])]).T
# print(arr_temp)
# # arr_temp[0:, :]
# # print(np.sum(arr_temp))

# # arr_temp[[2, 3], [1, 2]]
# arr_temp[np.ix_([2,3], [1,2])]

In [ ]:
alpha_init = np.zeros((num_edges, T))

VoT_in_boundary = np.zeros((num_edges, T))
VoT_el_boundary = np.zeros((num_edges, T))

for e in range(num_edges):
#     print("edge_to_od_dict[e]:", edge_to_od_dict[e])
    
    for t in range(T):
        
        ## Ineligible users:
        
        VoT_array_trunc_in = np.zeros((len(edge_to_od_dict[e]), len (in_indices)))
        demand_array_trunc_in = np.zeros((len(edge_to_od_dict[e]), len (in_indices)))
        
#         for od_index, od_temp in enumerate(edge_to_od_dict[e]):
#             for g_index, g_temp in enumerate(in_indices):
#                 VoT_array_trunc_in[od_index, g_index] = VoT_array[od_temp, g_temp, t] 
#                 demand_array_trunc_in[od_index, g_index] = demand_array[od_temp, g_temp] 
        
#         VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :, t]
#         VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices]
#         demand_array_trunc_in_temp = demand_array[edge_to_od_dict[e], :]
#         demand_array_trunc_in = demand_array_trunc_in_temp[:, in_indices]
        
        VoT_array_trunc_in = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], in_indices)]
        demand_array_trunc_in = demand_array[np.ix_(edge_to_od_dict[e], in_indices)]

#         print("np.sum(demand_array_trunc_in):", np.sum(demand_array_trunc_in))
#         print("(Same sum computed directly):", sum([demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                     for g in in_indices]))

        assert np.abs(np.sum(demand_array_trunc_in) - sum([demand_array[od, g] for od in edge_to_od_dict[e] \
                                                    for g in in_indices])) <= 1E-3
#         assert np.sum(demand_array_trunc_in_flattened) == sum([demand_array[od, g] for od in edge_to_od_dict[e] \
#                                                             for g in in_indices])
        
#         print("VoT_array_trunc_in.shape:", VoT_array_trunc_in.shape)
#         print("demand_array_trunc_in.shape:", demand_array_trunc_in.shape)
#         print("len([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices])", \
#               len([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices]))
#         print("demand_array_trunc_in:", demand_array_trunc_in)
#         print("[demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices]:", [demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices])

        VoT_array_trunc_in_flattened = VoT_array_trunc_in.flatten()
        demand_array_trunc_in_flattened = demand_array_trunc_in.flatten()
        VoT_demand_in = np.stack([VoT_array_trunc_in_flattened, demand_array_trunc_in_flattened]).T    
        VoT_demand_in_sorted = VoT_demand_in[np.argsort(VoT_demand_in[:, 0])]
        VoT_demand_in_cumul = np.zeros(VoT_demand_in_sorted.shape)
        VoT_demand_in_cumul[:, 0] = VoT_demand_in_sorted[:, 0]
        for row_index in range(VoT_demand_in_cumul.shape[0]):
            VoT_demand_in_cumul[row_index, 1] = np.sum(VoT_demand_in_sorted[row_index:, 1])
        
    
        
        assert np.abs(np.sum(VoT_demand_in[:, 1]) - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                            for g in in_indices)) <= 1E-3
        assert np.abs(np.sum(VoT_demand_in_sorted[:, 1]) - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                            for g in in_indices)) <= 1E-3
        assert np.abs(VoT_demand_in_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in in_indices)) <= 1E-3

        ## Eligible users:

        # Flatten VoT_array_trunc_in and demand_array_trunc_in, and concatenate.
        # Sort by VoT
        # Create cumulative demand array

#         VoT_array_trunc_el_temp = VoT_array[edge_to_od_dict[e], :, t]
#         VoT_array_trunc_el = VoT_array_trunc_el_temp[:, el_indices]
#         demand_array_trunc_el_temp = demand_array[edge_to_od_dict[e], :]
#         demand_array_trunc_el = demand_array_trunc_el_temp[:, el_indices]

        VoT_array_trunc_el = VoT_array[:, :, t][np.ix_(edge_to_od_dict[e], el_indices)]
        demand_array_trunc_el = demand_array[np.ix_(edge_to_od_dict[e], el_indices)]

        VoT_array_trunc_el_flattened = VoT_array_trunc_el.flatten()
        demand_array_trunc_el_flattened = demand_array_trunc_el.flatten()
        VoT_demand_el = np.stack([VoT_array_trunc_el_flattened, demand_array_trunc_el_flattened]).T    
        VoT_demand_el_sorted = VoT_demand_el[np.argsort(VoT_demand_el[:, 0])]
        VoT_demand_el_cumul = np.zeros(VoT_demand_el_sorted.shape)
        VoT_demand_el_cumul[:, 0] = VoT_demand_el_sorted[:, 0]
        for row_index in range(VoT_demand_el_cumul.shape[0]):
            VoT_demand_el_cumul[row_index, 1] = np.sum(VoT_demand_el_sorted[row_index:, 1])
    
        print("VoT_demand_el_cumul[0, 1]:", VoT_demand_el_cumul[0, 1])
        print("(Same sum computed directly):", sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in el_indices))
        print()
        assert np.abs(VoT_demand_el_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in el_indices)) <= 1E-3

    
#     VoT_array[od, g, t]


# for e in range(num_edges):
#     for t in range(T):
        
# for e in [0]:
#     for t in [0]:
        
        print()
        print("e:", e)
        print("t:", t)
        
        ## Ineligible users:
        
        # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
        # Find corresponding VoT index -> Value
        
        VoT_in_boundary_index = max([row_index for row_index in range(VoT_demand_in_cumul.shape[0]) \
                                     if VoT_demand_in_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'in')]])
        VoT_in_boundary[e, t] = VoT_demand_in_cumul[VoT_in_boundary_index, 0]
        
        ## Eligible users:
        
        # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
        # Find corresponding VoT index -> Value
        
        print("Total el user flow at e, t:", \
                 sum(demand_array[(od, g)] for od in edge_to_od_dict[e] for g in el_indices))
        print("argmin_y_in_el_total[(e, 0, t, 'el')]]:", argmin_y_in_el_total[(e, 0, t, 'el')])
        print("VoT_demand_el_cumul[:, 1]", VoT_demand_el_cumul[:, 1])
        print("VoT_demand_el_cumul[0, 1]:", VoT_demand_el_cumul[0, 1])
        print("(Same sum computed directly):", sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                                                for g in el_indices))
        print()
        assert np.abs(VoT_demand_el_cumul[0, 1] - sum(demand_array[od, g] for od in edge_to_od_dict[e] \
                        for g in el_indices)) <= 1E-3
        
        VoT_el_boundary_index = max([row_index for row_index in range(VoT_demand_el_cumul.shape[0]) \
                                     if VoT_demand_el_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'el')] - 1E-3])
        VoT_el_boundary[e, t] = VoT_demand_el_cumul[VoT_el_boundary_index, 0]
        
        if argmin_y_in_el_total[(e, 0, t, 'el')] <= 1E-3:
            alpha_init[e, t] = 0
        else:
            alpha_init[e, t] = 1.0 - VoT_el_boundary[e, t]/VoT_in_boundary[e, t]
        
        print("VoT_el_boundary_index:", VoT_el_boundary_index)
        print("VoT_in_boundary_index:", VoT_in_boundary_index)
        print("VoT_el_boundary[e, t]:", VoT_el_boundary[e, t])
        print("VoT_in_boundary[e, t]:", VoT_in_boundary[e, t])
        print("alpha_init[e, t]:", alpha_init[e, t])
        
        assert 0.0 <= alpha_init[e, t] <= 1.0


In [ ]:
# # VoT_array_trunc_in_flattened
# # demand_in_trunc_in_flattened

# print("argmin_y_in_el_total[(e, 0, t, el)]:\n", argmin_y_in_el_total[(e, 0, t, 'el')])
# print()
# print("argmin_y_in_el_total[(e, 0, t, in)]:\n", argmin_y_in_el_total[(e, 0, t, 'in')])
# print()
# print("VoT_demand_el_cumul:\n", VoT_demand_el_cumul)
# print()
# print("VoT_demand_in_cumul:\n", VoT_demand_in_cumul)



In [ ]:
# argmin_tau
# alpha_init

init_tau_alpha_array = np.block([argmin_tau, alpha_init])
init_tau_alpha_array

In [ ]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_inits_save = pd.DataFrame(init_tau_alpha_array, index=row_names, columns=column_names)

# df_inits_save

random_string = ""
for idx_rand_str in range(10):
    random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_tolls_subsidies_metrics/"
random_filename = "inits___" + random_string + '.csv'

df_inits_save.to_csv(directory_to_save + random_filename)

In [ ]:
# # argmin_y_in_el_total[(e, 0, t, 'in')]
# # demand_array_trunc_in_flattened

# for e in range(num_edges):
#     for t in range(T):
#         print("e:", e)
#         print("t:", t)
#         print("argmin_y_in_el_total[(e, 0, t, 'el')]:", argmin_y_in_el_total[(e, 0, t, 'el')])
#         print()

# Testing solve CBCP direct with a small example:

In [ ]:
# od_to_edges_array = np.array([[0, 0], [0, 1], [1, 1], [1, 2]])
# od_to_edges_list_full = [[0], [0, 1], [1], [1, 2]]
# edge_to_ods = [[0, 1], [1, 2, 3], [3]]

# num_od = 3
# num_edges = 3
# num_gp_lanes = 3
# num_groups = 5
# T = 5
# el_indices = [0, 1]
# in_indices = [2, 3, 4]
# B = 3
# tau = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
#         tau[e, t] = 1.5 + 0.5 * e + 0.2 * t

# demand_array = np.zeros((num_od, num_groups))
# VoT_array = np.zeros((num_od, num_groups, T))

# for od in range(num_od):
#     for g in range(num_groups):
#         demand_array[od, g] = 1.0 + od + 0.1 * g
#         VoT_array[od, g, t] = 2.0 + od + 0.1 * g
        
# coeff_input = np.array([1, 1, 0, 0, 0])
# latency_params_length = coeff_input.shape[0]

# ex_to_gp_multiplier = np.array([1/num_gp_lanes**p for p in range(latency_params_length)]).reshape((latency_params_length, 1)) \
#                         @ np.ones((1, num_edges))
# a = np.zeros((latency_params_length, num_edges, 2))
# coeff[:, :, 0] = coeff_input.reshape((latency_params_length, 1)) @ np.ones((1, num_edges))
# coeff[:, :, 1] = coeff[:, :, 0] * ex_to_gp_multiplier


# y_vals = solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                            demand_array, VoT_array, num_el, coeff_input)


# # y = {}
# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)

# # x = {}
# # for e in range(num_edges):
# #     for k in [0, 1]:
# #         for t in range(T):
# #             x[(e, k, t)] = cp.Variable(1)

# # # Objective:
# # func = 0.0
# # for e in range(num_edges):

# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
# #             for g in in_indices:
# #                 func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

# # objective = cp.Minimize(func)

# # # Constraints:
# # constraints = []

# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in el_indices  \
# #                 for k in [0, 1, 2] for t in range(T)]
# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in in_indices  \
# #                 for k in [0, 1] for t in range(T)]


# # for e in range(num_edges):
# #     for t in range(T):
# # #         print("e:", e)
# # #         print("edge_to_ods[e]:", edge_to_ods[e])

# #         ## Edge contraints:
# #         constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 0, t)] ]
# #         constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 1, t)] ]

# #         ## Group flow constraints:
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in el_indices]
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in in_indices]

# # constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
# #                 <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]

# # # Problem:
# # prob = cp.Problem(objective, constraints)

# # # Solve:
# # result = prob.solve()


# # assert prob.status != "infeasible", "problem.status should not be infeasible."
# # assert prob.status != "unbounded", "problem.status should not be unbounded."
# # print()
# # print("prob.status:", prob.status)

# # # Extract Values:
# # y_values = {}
# # for e in range(num_edges):
# #     for od in edge_to_ods[e]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)




In [ ]:
# demand_edges_array

In [ ]:
# np.sum(demand_array[3:16, :])/5

In [ ]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:
func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
